In [1]:
%load_ext autoreload
%autoreload 2

import pickle
import math

# The deformation module library is not automatically installed yet, we need to add its path manually
import sys
sys.path.append("../../")

import matplotlib.pyplot as plt
import torch

import implicitmodules.torch as dm

torch.set_default_tensor_type(torch.FloatTensor)

In [2]:
data = pickle.load(open("herd.pickle", 'rb'))

ear_pos_mean = list(torch.mean(torch.stack(list(zip(*list(zip(*data))[1]))[i], dim=0), dim=0) for i in range(2))
ear_pos_stdvar = list(torch.sqrt(torch.mean(torch.var(torch.stack(list(zip(*list(zip(*data))[1]))[i], dim=0), dim=0))) for i in range(2))

right_ear_tip_pos = data[0][1][0]
left_ear_tip_pos = data[0][1][1]
right_ear_tip_dir = right_ear_tip_pos/torch.norm(right_ear_tip_pos)
left_ear_tip_dir = left_ear_tip_pos/torch.norm(left_ear_tip_pos)

# We need a better template than that
template = data[0][0]
template = dm.Utilities.gaussian_kernel_smooth(template, 0.1)
herd = list(list(zip(*data))[0])[1:-1]

print(template.shape)
print(len(herd))

ear_sigma = 0.6

torch.Size([107, 2])
14


In [3]:
%matplotlib qt5
plt.plot(template[:, 0].numpy(), template[:, 1].numpy(), '--', color='black', lw=4.)
for bunny in herd:
    plt.plot(bunny.numpy()[:, 0], bunny.numpy()[:, 1], '-', color='C4')
# plt.plot(ear_pos_mean[0][0].numpy(), ear_pos_mean[0][1].numpy(), 'X')
# plt.plot(ear_pos_mean[1][0].numpy(), ear_pos_mean[1][1].numpy(), 'X')
plt.quiver(right_ear_tip_pos[0].numpy(), right_ear_tip_pos[1].numpy(), right_ear_tip_dir[0].numpy(), right_ear_tip_dir[1].numpy(), color='red')
plt.quiver(left_ear_tip_pos[0], left_ear_tip_pos[1], left_ear_tip_dir[0], left_ear_tip_dir[1], color='blue')
plt.axis('equal')
plt.show()


In [4]:
sigma_rotation = 4.
coeff_rotation = 0.005
coeff_translation = 0.5

right_ear_translation = dm.DeformationModules.OrientedTranslations(2, 1, ear_sigma, gd=(right_ear_tip_pos.clone().unsqueeze(0).requires_grad_(), right_ear_tip_dir.clone().unsqueeze(0).requires_grad_()), label='right_ear_translation')
left_ear_translation = dm.DeformationModules.OrientedTranslations(2, 1, ear_sigma, gd=(left_ear_tip_pos.clone().unsqueeze(0).requires_grad_(), left_ear_tip_dir.clone().unsqueeze(0).requires_grad_()), label='left_ear_translation')
#rigid_rotation = dm.DeformationModules.LinearDeformation.build(torch.tensor([[0., -1.], [1., 0.]]), gd=torch.tensor([[0., 0.]], requires_grad=True))
rigid_translation = dm.DeformationModules.GlobalTranslation(2, coeff_translation)
local_rotation = dm.DeformationModules.LocalRotation(2, sigma_rotation, coeff=coeff_rotation, gd=torch.tensor([[0., 0.]], requires_grad=True))

targets = herd

atlas = dm.Models.Atlas(template, [right_ear_translation, left_ear_translation, local_rotation, rigid_translation], [dm.Attachment.VarifoldAttachment(2, [0.3, 1.])], len(targets), fit_gd=None, lam=100.)

In [ ]:
fitter = dm.Models.ModelFittingScipy(atlas, 1.)
costs = fitter.fit(targets, 3000, options={'shoot_method': 'rk4'})

Initial energy = 54922.344


Iteration: 1 
Total energy = 50368.226562 
Attach cost = 50367.820312 
Deformation cost = 0.406464


Iteration: 2 
Total energy = 48056.859375 
Attach cost = 48051.246094 
Deformation cost = 5.613342


Iteration: 3 
Total energy = 40174.871094 
Attach cost = 40130.699219 
Deformation cost = 44.174484


Iteration: 4 
Total energy = 35535.484375 
Attach cost = 35463.296875 
Deformation cost = 72.185493


...

Iteration: 1081 
Total energy = 12701.965820 
Attach cost = 12604.240234 
Deformation cost = 97.724754


Iteration: 1082 
Total energy = 12700.525391 
Attach cost = 12602.503906 
Deformation cost = 98.020172


Iteration: 1083 
Total energy = 12699.488281 
Attach cost = 12601.333008 
Deformation cost = 98.155518


Iteration: 1084 
Total energy = 12699.043945 
Attach cost = 12600.744141 
Deformation cost = 98.298309


Iteration: 1085 
Total energy = 12697.756836 
Attach cost = 12599.287109 
Deformation cost = 98.469902


In [ ]:
%matplotlib qt5
plt.plot(range(len(costs)), costs)
plt.show()

In [8]:
print(atlas.models[0].init_manifold[1].gd)
print(atlas.models[0].init_manifold[2].gd)
print(atlas.models[0].init_manifold[3].gd)

(tensor([[0.6164, 1.4765]], requires_grad=True), tensor([[0.3852, 0.9228]], requires_grad=True))
(tensor([[-1.0290,  1.2253]], requires_grad=True), tensor([[-0.6431,  0.7658]], requires_grad=True))
tensor([[0., 0.]], requires_grad=True)


In [9]:
%matplotlib qt5
shoot_it = 10
shoot_method = 'rk4'
it_per_snapshot = 2
snapshots = int(shoot_it/it_per_snapshot)

disp_targets = len(targets)

for i in range(disp_targets):
    silent_pos = atlas.models[i].init_manifold[0].gd.detach().clone()
    silent_mom = atlas.models[i].init_manifold[0].cotan.detach().clone()
    right_ear_trans_gd = (atlas.models[i].init_manifold[1].gd[0].detach().clone(),
                          atlas.models[i].init_manifold[1].gd[1].detach().clone())
    right_ear_trans_mom = (atlas.models[i].init_manifold[1].cotan[0].detach().clone(),
                           atlas.models[i].init_manifold[1].cotan[1].detach().clone())
    left_ear_trans_gd = (atlas.models[i].init_manifold[2].gd[0].detach().clone(),
                          atlas.models[i].init_manifold[2].gd[1].detach().clone())
    left_ear_trans_mom = (atlas.models[i].init_manifold[2].cotan[0].detach().clone(),
                          atlas.models[i].init_manifold[2].cotan[1].detach().clone())
    local_rotation_gd = atlas.models[i].init_manifold[3].gd.detach().clone()
    local_rotation_mom = atlas.models[i].init_manifold[3].cotan.detach().clone()

    silent = dm.DeformationModules.SilentLandmarks(2, silent_pos.shape[0], gd=silent_pos, cotan=silent_mom)
    right_ear_trans = dm.DeformationModules.OrientedTranslations(2, 1, ear_sigma, gd=right_ear_trans_gd, cotan=right_ear_trans_mom)
    left_ear_trans = dm.DeformationModules.OrientedTranslations(2, 1, ear_sigma, gd=left_ear_trans_gd, cotan=left_ear_trans_mom)
    #global_trans = dm.DeformationModules.GlobalTranslation(2)
    local_rot = dm.DeformationModules.LocalRotation(2, sigma_rotation, coeff=coeff_rotation, gd=local_rotation_gd, cotan=local_rotation_mom)
    rigid_translation = dm.DeformationModules.GlobalTranslation(2)

    h = dm.HamiltonianDynamic.Hamiltonian([silent, right_ear_trans, left_ear_trans, local_rot, rigid_translation])
    intermediate_states, _ = dm.HamiltonianDynamic.shoot(h, shoot_it, shoot_method, intermediates=True)

    for j in range(snapshots):
        pos = intermediate_states[it_per_snapshot*j].gd[0].numpy()
        pos_right_ear_trans = intermediate_states[it_per_snapshot*j].gd[1][0].numpy()
        pos_left_ear_trans = intermediate_states[it_per_snapshot*j].gd[2][0].numpy()

        plt.subplot(disp_targets, snapshots + 1, i*snapshots + j + i + 1)
        plt.plot(pos[:, 0], pos[:, 1])
        plt.plot(pos_right_ear_trans[:, 0], pos_right_ear_trans[:, 1], 'x')
        plt.plot(pos_left_ear_trans[:, 0], pos_left_ear_trans[:, 1], 'x')
        plt.plot(targets[i].numpy()[:, 0], targets[i].numpy()[:, 1])
        plt.axis('equal')


    final_pos = intermediate_states[-1].gd[0].numpy()
    plt.subplot(disp_targets, snapshots + 1, i*snapshots + snapshots + i + 1)
    plt.plot(targets[i].numpy()[:, 0], targets[i].numpy()[:, 1])
    plt.plot(final_pos[:, 0], final_pos[:, 1])
    plt.axis('equal')

plt.show()